## Load Data from CSVs

#### This code was adjusted to run on Python 3

In [ ]:
import unicodecsv

## Longer version of code (replaced with shorter, equivalent version below)

# enrollments = []
# f = open('enrollments.csv', 'rb')
# reader = unicodecsv.DictReader(f)
# for row in reader:
#     enrollments.append(row)
# f.close()

with open('./datasets_lesson1/enrollments.csv', 'rb') as f:
    reader = unicodecsv.DictReader(f)
    enrollments = list(reader)

In [ ]:
#####################################
#                 1                 #
#####################################

## Read in the data from daily_engagement.csv and project_submissions.csv 
## and store the results in the below variables.
## Then look at the first row of each table.

enrollment_filename = './datasets_lesson1/enrollments.csv'
engagement_filename = './datasets_lesson1/daily_engagement.csv'
submissions_filename = './datasets_lesson1/project_submissions.csv'

In [ ]:
with open(engagement_filename, 'rb') as f:
    reader = unicodecsv.DictReader(f)
    daily_engagement = list(reader)

with open(submissions_filename, 'rb') as f:
    reader = unicodecsv.DictReader(f)
    project_submissions = list(reader)

# check if files are loaded properly
print(daily_engagement[0])
print('---------------')
print(project_submissions[0])

In [ ]:
# function for loading csv files

def read_csv(filename):
    with open(filename, 'rb') as f:
        reader = unicodecsv.DictReader(f)
        return list(reader)

enrollments = read_csv(enrollment_filename)
print('enrollments loaded succesfully')
daily_engagement = read_csv(engagement_filename)
print('daily engagement loaded succesfully')
project_submissions = read_csv(submissions_filename)
print('submissions loaded succesfully')

In [ ]:
enrollments[0]

In [ ]:
daily_engagement[0]

In [ ]:
project_submissions[0]

## Fixing Data Types

In [ ]:
from datetime import datetime as dt

# Takes a date as a string, and returns a Python datetime object. 
# If there is no date given, returns None
def parse_date(date):
    if date == '':
        return None
    else:
        return dt.strptime(date, '%Y-%m-%d')
    
# Takes a string which is either an empty string or represents an integer,
# and returns an int or None.
def parse_maybe_int(i):
    if i == '':
        return None
    else:
        return int(i)

# Clean up the data types in the enrollments table
for enrollment in enrollments:
    enrollment['cancel_date'] = parse_date(enrollment['cancel_date'])
    enrollment['days_to_cancel'] = parse_maybe_int(enrollment['days_to_cancel'])
    enrollment['is_canceled'] = enrollment['is_canceled'] == 'True'
    enrollment['is_udacity'] = enrollment['is_udacity'] == 'True'
    enrollment['join_date'] = parse_date(enrollment['join_date'])
    
enrollments[0]

In [ ]:
# Clean up the data types in the engagement table
for engagement_record in daily_engagement:
    engagement_record['lessons_completed'] = int(float(engagement_record['lessons_completed']))
    engagement_record['num_courses_visited'] = int(float(engagement_record['num_courses_visited']))
    engagement_record['projects_completed'] = int(float(engagement_record['projects_completed']))
    engagement_record['total_minutes_visited'] = float(engagement_record['total_minutes_visited'])
    engagement_record['utc_date'] = parse_date(engagement_record['utc_date'])
    
daily_engagement[0]

In [ ]:
# Clean up the data types in the submissions table
for submission in project_submissions:
    submission['completion_date'] = parse_date(submission['completion_date'])
    submission['creation_date'] = parse_date(submission['creation_date'])

project_submissions[0]

## Investigating the Data

In [ ]:
#####################################
#                 2                 #
#####################################

## Find the total number of rows and the number of unique students (account keys)
## in each table.
# def unique_students(data):
#     return len(set([key for row in data for key in row.keys()]))

enrollment_num_rows = len(enrollments)            
enrollment_num_unique_students = set()
for enrollment in enrollments:
    enrollment_num_unique_students.add(enrollment['account_key'])
enrollment_num_unique_students = len(enrollment_num_unique_students)

engagement_num_rows = len(daily_engagement)
engagement_num_unique_students = set()
for record in daily_engagement:
    engagement_num_unique_students.add(record['acct'])
engagement_num_unique_students = len(engagement_num_unique_students)

submission_num_rows = len(project_submissions)
submission_num_unique_students = set()
for submission in project_submissions:
    submission_num_unique_students.add(submission['account_key'])
submission_num_unique_students = len(submission_num_unique_students)

print(enrollment_num_unique_students)
print(engagement_num_unique_students)
print(submission_num_unique_students)

## Problems in the Data

In [ ]:
#####################################
#                 3                 #
#####################################

## Rename the "acct" column in the daily_engagement table to "account_key".
for record in daily_engagement:
    record['account_key'] = record['acct']
    del record['acct']

In [ ]:
# function for retrieving unique students based account_key
def get_unique_students(data):
    unique_students = set()
    for data_point in data:
        unique_students.add(data_point['account_key'])
    return unique_students

enrollment_num_unique_students = len(get_unique_students(enrollments))
print(enrollment_num_unique_students)

## Missing Engagement Records

In [ ]:
#####################################
#                 4                 #
#####################################

## Find any one student enrollments where the student is missing from the daily engagement table.
## Output that enrollment.
for student in enrollments:
    enrollment_student = student['account_key']
    if enrollment_student not in get_unique_students(daily_engagement):
        print(student)
        break

## Checking for More Problem Records

In [ ]:
#####################################
#                 5                 #
#####################################

## Find the number of surprising data points (enrollments missing from
## the engagement table) that remain, if any.
missing_points = 0
for student in enrollments:
    enrollment_student = student['account_key']
    if enrollment_student not in get_unique_students(daily_engagement) and (student['join_date'] != student['cancel_date']):
            missing_points += 1
print(missing_points)

## Tracking Down the Remaining Problems

In [ ]:
# Create a set of the account keys for all Udacity test accounts
udacity_test_accounts = set()
for enrollment in enrollments:
    if enrollment['is_udacity']:
        udacity_test_accounts.add(enrollment['account_key'])
len(udacity_test_accounts)

In [ ]:
# Given some data with an account_key field, removes any records corresponding to Udacity test accounts
def remove_udacity_accounts(data):
    non_udacity_data = []
    for data_point in data:
        if data_point['account_key'] not in udacity_test_accounts:
            non_udacity_data.append(data_point)
    return non_udacity_data

In [ ]:
# Remove Udacity test accounts from all three tables
non_udacity_enrollments = remove_udacity_accounts(enrollments)
non_udacity_engagement = remove_udacity_accounts(daily_engagement)
non_udacity_submissions = remove_udacity_accounts(project_submissions)

print(len(non_udacity_enrollments))
print(len(non_udacity_engagement))
print(len(non_udacity_submissions))

## Refining the Question

In [ ]:
#####################################
#                 6                 #
#####################################

## Create a dictionary named paid_students containing all students who either
## haven't canceled yet or who remained enrolled for more than 7 days. The keys
## should be account keys, and the values should be the date the student enrolled.

paid_students = dict()
for enrollment in non_udacity_enrollments:
    if enrollment['days_to_cancel'] is None or enrollment['days_to_cancel'] > 7: 
        account_key = enrollment['account_key']
        enrollment_date = enrollment['join_date']
        
        if account_key not in paid_students or enrollment_date > paid_students[account_key]:
            paid_students[account_key] = enrollment_date
print(len(paid_students.keys()))

## Getting Data from First Week

In [ ]:
# Takes a student's join date and the date of a specific engagement record,
# and returns True if that engagement record happened within one week
# of the student joining.
def within_one_week(join_date, engagement_date):
    time_delta = engagement_date - join_date
    return time_delta.days >= 0 and time_delta.days < 7 

In [ ]:
#####################################
#                 7                 #
#####################################

## Create a list of rows from the engagement table including only rows where
## the student is one of the paid students you just found, and the date is within
## one week of the student's join date.

paid_engagement_in_first_week = list()
for record in daily_engagement:
    account_key = record['account_key']
    if account_key in paid_students.keys() and within_one_week(paid_students[account_key], record['utc_date']):
        paid_engagement_in_first_week.append(record)
print(len(paid_engagement_in_first_week))

In [ ]:
def remove_free_trial_cancels(data):
    new_data = list()
    for data_point in data:
        if data_point['account_key'] in paid_students:
            new_data.append(data_point)
    return new_data

In [ ]:
paid_enrollments = remove_free_trial_cancels(non_udacity_enrollments)
paid_engagement = remove_free_trial_cancels(non_udacity_engagement)
paid_submissions = remove_free_trial_cancels(non_udacity_submissions)
print(len(paid_enrollments))
print(len(paid_engagement))
print(len(paid_submissions))

In [ ]:
# add has visited field to paid_engagement in order to sum up the number of visited courses in the first week
for engagement in paid_engagement:
    if engagement['num_courses_visited'] > 0:
        engagement['has_visited'] = 1
    else:
        engagement['has_visited'] = 0 

## Exploring Student Engagement

In [ ]:
from collections import defaultdict

# Create a dictionary of engagement grouped by student.
# The keys are account keys, and the values are lists of engagement records.
engagement_by_account = defaultdict(list)
for engagement_record in paid_engagement_in_first_week:
    account_key = engagement_record['account_key']
    engagement_by_account[account_key].append(engagement_record)

In [ ]:
# Create a dictionary with the total minutes each student spent in the classroom during the first week.
# The keys are account keys, and the values are numbers (total minutes)
total_minutes_by_account = {}
for account_key, engagement_for_student in engagement_by_account.items():
    total_minutes = 0
    for engagement_record in engagement_for_student:
        total_minutes += engagement_record['total_minutes_visited']
    total_minutes_by_account[account_key] = total_minutes

In [ ]:
import numpy as np

# Summarize the data about minutes spent in the classroom
total_minutes = list(total_minutes_by_account.values())
print('Mean:', np.mean(total_minutes))
print('Standard deviation:', np.std(total_minutes))
print('Minimum:', np.min(total_minutes))
print('Maximum:', np.max(total_minutes))

## Debugging Data Analysis Code

In [ ]:
#####################################
#                 8                 #
#####################################

## Go through a similar process as before to see if there is a problem.
## Locate at least one surprising piece of data, output it, and take a look at it.
student_with_max_minutes = 0
max_minutes = 0
for student, total_minutes in total_minutes_by_account.items():
    if total_minutes > max_minutes:
        max_minutes = total_minutes
        student_with_max_minutes = student
        
max_minutes

In [ ]:
for engagement in paid_engagement_in_first_week:
    if engagement['account_key'] == student_with_max_minutes:
        print(engagement)

## Lessons Completed in First Week

In [ ]:
#####################################
#                 9                 #
#####################################

## Adapt the code above to find the mean, standard deviation, minimum, and maximum for
## the number of lessons completed by each student during the first week. Try creating
## one or more functions to re-use the code above.
from collections import defaultdict

def group_data(data, key_name):
    grouped_data = defaultdict(list)
    for data_point in data:
        account_key = data_point[key_name]
        grouped_data[account_key].append(data_point)
    return grouped_data

engagement_by_account = group_data(paid_engagement_in_first_week, 'account_key')

In [ ]:
def sum_grouped_items(grouped_data, field_name):
    summed_data = {}
    for key, data_points in grouped_data.items():
        total = 0
        for data_point in data_points:
            total += data_point[field_name]
        summed_data[key] = total
    return summed_data

total_lessons_by_account = sum_grouped_items(engagement_by_account, 'lessons_completed')

In [ ]:
def summarize_data(data):
    total = list(data)
    print('Mean:', np.mean(total))
    print('Standard deviation:', np.std(total))
    print('Minimum:', np.min(total))
    print('Maximum:', np.max(total))

summarize_data(total_lessons_by_account.values())

## Number of Visits in First Week

In [ ]:
######################################
#                 10                 #
######################################

## Find the mean, standard deviation, minimum, and maximum for the number of
## days each student visits the classroom during the first week.
days_visited_by_account = sum_grouped_items(engagement_by_account, 'has_visited')
summarize_data(days_visited_by_account.values())

## Splitting out Passing Students

In [ ]:
######################################
#                 11                 #
######################################

## Create two lists of engagement data for paid students in the first week.
## The first list should contain data for students who eventually pass the
## subway project, and the second list should contain data for students
## who do not.

subway_project_lesson_keys = ['746169184', '3176718735']
pass_subway_project = set()
for submission in paid_submissions:
    project = submission['lesson_key']
    rating = submission['assigned_rating']
    if project in subway_project_lesson_keys and (rating == 'PASSED' or rating == 'DISTINCTION'):
        pass_subway_project.add(submission['account_key'])

len(pass_subway_project)

In [ ]:
passing_engagement = []
non_passing_engagement = []

for engagement in paid_engagement_in_first_week:
    if engagement['account_key'] in pass_subway_project:
        passing_engagement.append(engagement)
    else:
        non_passing_engagement.append(engagement)

print(len(passing_engagement))
print(len(non_passing_engagement))

## Comparing the Two Student Groups

In [ ]:
######################################
#                 12                 #
######################################

## Compute some metrics you're interested in and see how they differ for
## students who pass the subway project vs. students who don't. A good
## starting point would be the metrics we looked at earlier (minutes spent
## in the classroom, lessons completed, and days visited).

# Lessons completed

lessons_completed_by_passing_students = group_data(passing_engagement, 'account_key')
grouped_lessons_completed_by_passing_students = sum_grouped_items(lessons_completed_by_passing_students, 'lessons_completed')
summarize_data(grouped_lessons_completed_by_passing_students.values())
print('-----')
lessons_completed_by_nonpassing_students = group_data(non_passing_engagement, 'account_key')
grouped_lessons_completed_by_nonpassing_students = sum_grouped_items(lessons_completed_by_nonpassing_students, 'lessons_completed')
summarize_data(grouped_lessons_completed_by_nonpassing_students.values())

In [ ]:
# projects_completed
projects_completed_by_passing_students = group_data(passing_engagement, 'account_key')
grouped_projects_completed_by_passing_students = sum_grouped_items(projects_completed_by_passing_students, 'projects_completed')
summarize_data(grouped_projects_completed_by_passing_students.values())
print('-----')
projects_completed_by_nonpassing_students = group_data(non_passing_engagement, 'account_key')
grouped_projects_completed_by_nonpassing_students = sum_grouped_items(projects_completed_by_nonpassing_students, 'projects_completed')
summarize_data(grouped_projects_completed_by_nonpassing_students.values())

In [ ]:
# days visited
days_visited_by_passing_students = group_data(passing_engagement, 'account_key')
grouped_days_visited_by_passing_students = sum_grouped_items(days_visited_by_passing_students, 'has_visited')
summarize_data(grouped_days_visited_by_passing_students.values())
print('-----')
days_visited_by_nonpassing_students = group_data(non_passing_engagement, 'account_key')
grouped_days_visited_by_nonpassing_students = sum_grouped_items(days_visited_by_nonpassing_students, 'has_visited')
summarize_data(grouped_days_visited_by_nonpassing_students.values())

In [ ]:
# Minutes spent
print('passing students')
minutes_spent_by_passing_students = group_data(passing_engagement, 'account_key')
grouped_minutes_spent_by_passing_students = sum_grouped_items(minutes_spent_by_passing_students, 'total_minutes_visited')
summarize_data(grouped_minutes_spent_by_passing_students.values())
print('-----')
print('non-passing students')
minutes_spent_by_nonpassing_students = group_data(non_passing_engagement, 'account_key')
grouped_minutes_spent_by_nonpassing_students = sum_grouped_items(minutes_spent_by_nonpassing_students, 'total_minutes_visited')
summarize_data(grouped_minutes_spent_by_nonpassing_students.values())

## Making Histograms

In [ ]:
######################################
#                 13                 #
######################################

## Make histograms of the three metrics we looked at earlier for both
## students who passed the subway project and students who didn't. You
## might also want to make histograms of any other metrics you examined.
%matplotlib inline
import matplotlib.pyplot as plt

plt.hist(grouped_days_visited_by_passing_students.values())

In [ ]:
plt.hist(grouped_days_visited_by_nonpassing_students.values())

In [ ]:
plt.hist(grouped_lessons_completed_by_nonpassing_students.values())

In [ ]:
plt.hist(grouped_lessons_completed_by_passing_students.values())

In [ ]:
plt.hist(grouped_minutes_spent_by_nonpassing_students.values())

In [ ]:
plt.hist(grouped_minutes_spent_by_passing_students.values())

## Improving Plots and Sharing Findings

In [ ]:
######################################
#                 14                 #
######################################

## Make a more polished version of at least one of your visualizations
## from earlier. Try importing the seaborn library to make the visualization
## look better, adding axis labels and a title, and changing one or more
## arguments to the hist() function.
import seaborn as sns

#plt.hist(grouped_minutes_spent_by_passing_students.values(), bins=20)
sns.distplot(list(grouped_minutes_spent_by_passing_students.values()))

In [ ]:
sns.distplot(list(grouped_minutes_spent_by_nonpassing_students.values()))